In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

zip_path = "/content/drive/MyDrive/hsi_data_pca.zip"
!unzip -o "{zip_path}" -d /content/hsi_data
!ls -lh /content/hsi_data/preprocessedd

Mounted at /content/drive
Archive:  /content/drive/MyDrive/hsi_data_pca.zip
   creating: /content/hsi_data/preprocessed/
  inflating: /content/hsi_data/preprocessed/y_train.npy  
  inflating: /content/hsi_data/preprocessed/y_test.npy  
  inflating: /content/hsi_data/preprocessed/X_test.npy  
  inflating: /content/hsi_data/preprocessed/X_train.npy  
   creating: /content/hsi_data/preprocessedd/
  inflating: /content/hsi_data/preprocessedd/y_train.npy  
  inflating: /content/hsi_data/preprocessedd/y_test.npy  
  inflating: /content/hsi_data/preprocessedd/X_test.npy  
  inflating: /content/hsi_data/preprocessedd/X_train.npy  
total 734M
-rw-r--r-- 1 root root 661M Sep  8 10:24 X_test.npy
-rw-r--r-- 1 root root  73M Sep  8 10:24 X_train.npy
-rw-r--r-- 1 root root  73K Sep  8 10:24 y_test.npy
-rw-r--r-- 1 root root 8.1K Sep  8 10:24 y_train.npy


In [5]:
import numpy as np

X_train = np.load("/content/hsi_data/preprocessedd/X_train.npy")
X_test  = np.load("/content/hsi_data/preprocessedd/X_test.npy")
y_train = np.load("/content/hsi_data/preprocessedd/y_train.npy")
y_test  = np.load("/content/hsi_data/preprocessedd/y_test.npy")

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(1018, 25, 25, 30) (9231, 25, 25, 30)
(1018,) (9231,)


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [21]:
class HSIDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Load data
X_train = np.load("/content/hsi_data/preprocessedd/X_train.npy")
y_train = np.load("/content/hsi_data/preprocessedd/y_train.npy")
X_test  = np.load("/content/hsi_data/preprocessedd/X_test.npy")
y_test  = np.load("/content/hsi_data/preprocessedd/y_test.npy")

# Create datasets
train_dataset = HSIDataset(X_train, y_train)
test_dataset  = HSIDataset(X_test, y_test)

# Create loaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train samples:", len(train_dataset), "Test samples:", len(test_dataset))

Train samples: 1018 Test samples: 9231


In [26]:
# 3D CNN branch
class Branch3DConv(nn.Module):
    def __init__(self, in_bands, out_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(7,3,3), padding=(3,1,1)),
            nn.BatchNorm3d(8), nn.ReLU(),
            nn.Conv3d(8, 16, kernel_size=(5,3,3), padding=(2,1,1)),
            nn.BatchNorm3d(16), nn.ReLU(),
            nn.AdaptiveAvgPool3d(1)
        )
        self.fc = nn.Linear(16, out_dim)
        self.out_dim = out_dim

    def forward(self, x):
        x = x.unsqueeze(1)  # [B,1,bands,H,W]
        x = self.conv(x).view(x.size(0), -1)
        return self.fc(x)

# 1D wavelet branch
class BranchHaar1D(nn.Module):
    def __init__(self, in_bands, out_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.Conv1d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(16, out_dim)
        self.out_dim = out_dim

    def forward(self, x):
        x = x.mean(-1).mean(-1).unsqueeze(1)  # [B,1,bands]
        x = self.conv(x).view(x.size(0), -1)
        return self.fc(x)

# 2D wavelet branch
class BranchHaar2D(nn.Module):
    def __init__(self, out_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(16, out_dim)
        self.out_dim = out_dim

    def forward(self, x):
        x = x.mean(1, keepdim=True)  # collapse bands -> [B,1,H,W]
        x = self.conv(x).view(x.size(0), -1)
        return self.fc(x)

# 3D wavelet branch
class BranchHaar3D(nn.Module):
    def __init__(self, in_bands, out_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(8, 16, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d(1)
        )
        self.fc = nn.Linear(16, out_dim)
        self.out_dim = out_dim

    def forward(self, x):
        x = x.unsqueeze(1)  # [B,1,bands,H,W]
        x = self.conv(x).view(x.size(0), -1)
        return self.fc(x)

In [27]:
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channel, channel // reduction)
        self.fc2 = nn.Linear(channel // reduction, channel)

    def forward(self, x):
        # x: [B, C]
        y = x.mean(dim=0, keepdim=True)  # squeeze globally
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y))
        return x * y  # scale features

In [28]:
class MultiBranchModelSE(nn.Module):
    def __init__(self, in_bands, num_classes, out_dim=128):
        super().__init__()
        self.branch1 = Branch3DConv(in_bands, out_dim)
        self.branch2 = BranchHaar1D(in_bands, out_dim)
        self.branch3 = BranchHaar2D(out_dim)
        self.branch4 = BranchHaar3D(in_bands, out_dim)

        # SE Blocks
        self.se1 = SEBlock(out_dim)
        self.se2 = SEBlock(out_dim)
        self.se3 = SEBlock(out_dim)
        self.se4 = SEBlock(out_dim)

        # Fusion
        self.fc1 = nn.Linear(out_dim*4, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        b1 = self.se1(self.branch1(x))
        b2 = self.se2(self.branch2(x))
        b3 = self.se3(self.branch3(x))
        b4 = self.se4(self.branch4(x))

        fused = torch.cat([b1, b2, b3, b4], dim=1)
        out = F.relu(self.fc1(fused))
        return self.fc2(out)

In [29]:
in_bands = X_train.shape[1]  # number of bands
num_classes = 16

model = MultiBranchModelSE(in_bands, num_classes).to(device)

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Assuming: model, train_loader, device are already defined
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 200
patience = 10       # early stopping patience
best_loss = float('inf')
counter = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    loop = tqdm(train_loader, leave=False)

    for X_batch, y_batch in loop:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y_batch.size(0)
        correct += predicted.eq(y_batch).sum().item()

        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(loss=running_loss/len(train_loader), acc=100*correct/total)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Early stopping based on training loss
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")  # save best model
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

print("Training complete!")

Epoch 1 Loss: 0.6253, Accuracy: 78.49%


Epoch 2 Loss: 0.5615, Accuracy: 80.06%


Epoch 3 Loss: 0.5618, Accuracy: 79.27%


Epoch 4 Loss: 0.5697, Accuracy: 79.76%


Epoch 5 Loss: 0.5174, Accuracy: 81.34%


...

Epoch 70 Loss: 0.1144, Accuracy: 96.07%


Epoch 71 Loss: 0.1045, Accuracy: 95.58%


Epoch 72 Loss: 0.1072, Accuracy: 95.87%


Epoch 73 Loss: 0.1203, Accuracy: 95.28%


Epoch 74 Loss: 0.1293, Accuracy: 95.28%
Early stopping triggered at epoch 74
Training complete!


In [34]:
from sklearn.metrics import classification_report, accuracy_score
import torch

# Make sure model is in evaluation mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:  # your test_loader
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# Overall test accuracy
test_acc = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Detailed class-wise report
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds))

Test Accuracy: 89.18%

Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.83      0.88        42
           1       0.81      0.84      0.83      1286
           2       0.85      0.92      0.88       747
           3       0.86      0.86      0.86       214
           4       0.90      0.88      0.89       435
           5       0.81      0.98      0.89       657
           6       0.39      0.85      0.54        26
           7       0.97      1.00      0.98       431
           8       0.31      0.28      0.29        18
           9       0.96      0.93      0.95       875
          10       0.99      0.82      0.90      2210
          11       0.83      0.89      0.86       534
          12       0.92      0.64      0.75       185
          13       0.95      1.00      0.97      1139
          14       0.82      0.97      0.88       348
          15       0.59      0.89      0.71        84

    accuracy                     

In [35]:
import numpy as np

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

num_classes = len(np.unique(all_labels))
class_acc = []

for i in range(num_classes):
    idx = all_labels == i
    acc = (all_preds[idx] == all_labels[idx]).sum() / idx.sum()
    class_acc.append(acc)
    print(f"Class {i}: Accuracy = {acc*100:.2f}%")

Class 0: Accuracy = 83.33%
Class 1: Accuracy = 84.21%
Class 2: Accuracy = 91.57%
Class 3: Accuracy = 86.45%
Class 4: Accuracy = 87.59%
Class 5: Accuracy = 97.87%
Class 6: Accuracy = 84.62%
Class 7: Accuracy = 100.00%
Class 8: Accuracy = 27.78%
Class 9: Accuracy = 93.14%
Class 10: Accuracy = 81.63%
Class 11: Accuracy = 89.14%
Class 12: Accuracy = 63.78%
Class 13: Accuracy = 100.00%
Class 14: Accuracy = 96.55%
Class 15: Accuracy = 89.29%
